# Notebook for displaying results tables
Note: these will only run once you've run `test.py` on the corresponding test sets,
e.g. see `scripts/04_eval_checkpoint.sh`, as it pulls the numbers from the respective experiment directories. 

To run the face forensics tables, you'll need to have processed the face forensics dataset accordingly, following `scripts/00_data_processing_faceforensics_aligned_frames.sh`, and then evalute the corresponding experiments in `scripts/04_eval_checkpoint.sh`.

In [ ]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
pd.options.display.float_format = '{:,.2f}'.format # print 2 decimal places

In [ ]:
dataset_names_list = {
    'gen_models': ['celebahq-pgan-pretrained', 'celebahq-sgan-pretrained',
                   'celebahq-glow-pretrained', 'celeba-gmm', 'ffhq-pgan', 
                   'ffhq-sgan', 'ffhq-sgan2'],
    'faceforensics': ['DF', 'NT', 'F2F', 'FS'],
}

In [ ]:
def make_table(model_paths, model_names, dataset_names, ens_type, partition='val', epoch='epoch_bestval'):
    # ens types: voted, avg_before_softmax, avg_after_softmax
    metrics_ap = dict()
    metrics_acc = dict()
    counts = dict()
    for model_path, model_name in zip(model_paths, model_names):
        metrics_ap[model_name]  = []
        metrics_acc[model_name] = []
        counts[model_name] = []
        for dataset in dataset_names:
            metric_file = os.path.join(model_path, partition, epoch, dataset, 'metrics_%s.npz' % ens_type)
            if os.path.isfile(metric_file):
                metrics = np.load(metric_file)
                metrics_ap[model_name].append(metrics['ap'])
                metrics_acc[model_name].append(metrics['acc'])
                counts[model_name].append(metrics['n'])
            else:
                metric_file = os.path.join(model_path, partition, epoch, dataset, 'metrics.npz')
                if os.path.isfile(metric_file):
                    # print("Using metrics.npz for %s" % model_name)
                    metrics = np.load(metric_file)
                    metrics_ap[model_name].append(metrics['ap'])
                    metrics_acc[model_name].append(metrics['acc'])
                    counts[model_name].append(metrics['n'])
                else:
                    metrics_ap[model_name].append(np.nan)
                    metrics_acc[model_name].append(np.nan)
                    counts[model_name].append(0)
    df_ap = pd.DataFrame(data=metrics_ap, index=dataset_names).T * 100
    df_acc = pd.DataFrame(data=metrics_acc, index=dataset_names).T * 100
    df_counts = pd.DataFrame(data=counts, index=dataset_names).T
    # df_counts is for sanity checking the number of images processed
    return df_ap, df_acc, df_counts

In [ ]:
def get_listing_for_group(grp_string):
    # grp_string = gp1 etc
    expts = sorted(glob.glob('../results/%s-*block*' % grp_string))
    names = ['-'.join(x.split('_')[-4:-2]) for x in expts]
    baseline_expts = sorted(glob.glob('../results/%s-*baseline*' % grp_string))
    baseline_names = ['-'.join(x.split('_')[-2:]) for x in baseline_expts]
    # print(baseline_expts)
    # print(baseline_names)
    
    if baseline_expts:
        order = [4,0,3,5,1,2]
        baseline_expts = [baseline_expts[i] for i in order]
        baseline_names = [baseline_names[i] for i in order]
    else:
        print("No baselines found")
        
    return expts+baseline_expts, names+baseline_names

###  xception gen models tables

In [ ]:
expts, names = get_listing_for_group('gp1')
t1, t2, n = make_table(expts, names, dataset_names_list['gen_models'], 'avg_after_softmax', partition='test')
display(t1)

In [ ]:
# random crop
expts, names = get_listing_for_group('gp1b') # random crop
t1, t2, n = make_table(expts, names, dataset_names_list['gen_models'], 'avg_after_softmax', partition='test')
display(t1)

# random resize crop
expts, names = get_listing_for_group('gp1a') # random resize crop
t1, t2, n = make_table(expts, names, dataset_names_list['gen_models'], 'avg_after_softmax', partition='test')
display(t1)


In [ ]:
# samples only
expts, names = get_listing_for_group('gp1d') # samples only
t1, t2, n = make_table(expts, names, dataset_names_list['gen_models'], 'avg_after_softmax', partition='test')
display(t1)

# inverses only
expts, names = get_listing_for_group('gp1c') # inverse only
t1, t2, n = make_table(expts, names, dataset_names_list['gen_models'], 'avg_after_softmax', partition='test')
display(t1)

# face forensics

In [ ]:
expts, names = get_listing_for_group('gp5') # train on F2F
t1, t2, n = make_table(expts, names, dataset_names_list['faceforensics'], 'avg_after_softmax', 'test')
display(t1)

In [ ]:
expts, names = get_listing_for_group('gp4') # train on FS
t1, t2, n = make_table(expts, names, dataset_names_list['faceforensics'], 'avg_after_softmax', 'test')
display(t1)

In [ ]:
expts, names = get_listing_for_group('gp3') # train on NT
t1, t2, n = make_table(expts, names, dataset_names_list['faceforensics'], 'avg_after_softmax', 'test')
display(t1)
table_nt = t1

In [ ]:
expts, names = get_listing_for_group('gp2') # train on DF
t1, t2, n = make_table(expts, names, dataset_names_list['faceforensics'], 'avg_after_softmax', 'test')
display(t1)